# 第5章 深度学习计算

## 5.1 层和块

### 练习 5.1.1 

如果将MySequential中存储块的方式更改为Python列表，会出现什么样的问题？ 

**解答：** 

&emsp;&emsp;如果将MySequential中存储块的方式从OrderedDict更改为Python列表，代码可以正常计算，但并没有注册给Module。无法像`_modules`一样使用很多内置方完成已经实现的功能。如无法通过`net.state_dict()`访问模型的网络结构和参数。

In [1]:
import torch.nn as nn
import torch

class MySequential(nn.Module):
    # 使用OrderedDict存储块的类
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

class MySequential_list(nn.Module):
    # 使用list存储块的类
    def __init__(self, *args):
        super(MySequential_list, self).__init__()
        self.sequential = []
        for module in args:
            self.sequential.append(module)

    def forward(self, X):
        for module in self.sequential:
            X = module(X)
        return X


X = torch.rand(1,10)
# 初始化两个block，确保传给MySequential和MySequential_list的是一样的参数。消除nn.Linear初始化时生成随机参数对结果的影响。
block1 = nn.Linear(10, 20)
block2 = nn.Linear(20, 10)
net = MySequential(block1, nn.ReLU(), block2)
net_list = MySequential_list(block1, nn.ReLU(), block2)

&emsp;&emsp;对比两种方式的结果，可以发现输出完全一致。

In [2]:
net_y = net(X)
net_list_y = net_list(X)

print("采用python OrderedDict的方式存储 :", net_y, sep='\n')
print("\n")
print("采用python列表的方式存储 :", net_list_y, sep='\n')
print("\n")
print("两种方式的计算结果是否一致：", net_y.equal(net_list_y))

采用python OrderedDict的方式存储 :
tensor([[0.0582, 0.1493, 0.0690, 0.4035, 0.1546, 0.1691, 0.0714, 0.2457, 0.0915,
         0.2701]], grad_fn=<AddmmBackward0>)


采用python列表的方式存储 :
tensor([[0.0582, 0.1493, 0.0690, 0.4035, 0.1546, 0.1691, 0.0714, 0.2457, 0.0915,
         0.2701]], grad_fn=<AddmmBackward0>)


两种方式的计算结果是否一致： True


&emsp;&emsp;但是在查看模型结构和参数时存在差异。

In [3]:
# 使用_modules方便打印net的网络结构和参数，而list则无法做到
print("采用python OrderedDict的方式存储 :\n")
print("网络结构 : \n", net)
# 为了可视化效果简洁，这里只展示了每个参数的部分值
print("参数 ：", "\n".join([str(k) + str(" ") + str(v[:2]) for k,v in net.state_dict().items()]), sep='\n')
print("\n")
print("采用python列表的方式存储 :\n")
print("网络结构 : \n", net) 
print("参数：", net_list.state_dict())

采用python OrderedDict的方式存储 :

网络结构 : 
 MySequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
)
参数 ：
0.weight tensor([[-0.1362, -0.1046,  0.2931,  0.1037, -0.0262,  0.0397,  0.3039,  0.2889,
         -0.2657, -0.2991],
        [-0.1937,  0.1747,  0.1011, -0.2762, -0.2860,  0.1979, -0.2156, -0.2557,
          0.2605, -0.0732]])
0.bias tensor([-0.2270, -0.0875])
2.weight tensor([[ 0.0352,  0.1039, -0.2078, -0.0484, -0.1247,  0.0517,  0.1218, -0.0847,
         -0.0480,  0.0699,  0.1584,  0.0302,  0.1800,  0.0026, -0.2152, -0.2162,
          0.1819,  0.0288, -0.0321, -0.1135],
        [ 0.0210, -0.1428, -0.0452,  0.0554,  0.1378,  0.0806,  0.0480,  0.1631,
          0.0599, -0.1287,  0.0318, -0.2155,  0.1711,  0.2007,  0.1199, -0.0329,
         -0.1352, -0.0747,  0.1737,  0.0999]])
2.bias tensor([ 0.1084, -0.0108])


采用python列表的方式存储 :

网络结构 : 
 MySequential(
  (0): Linear(in_features=10, out_features=20

### 练习 5.1.2

实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的并联输出。这也被称为并行块。

**解答：**

&emsp;&emsp;在本书7.4节中`GoogleLet`模型中的`Inception`块使用了并行块技术。
下面代码实现了一个并行网络，由两个子网络组成。输入数据先分别经过两个子网络的计算，分别得到两个部分的输出结果，然后在通道维度上合并结果得到最终输出。

&emsp;&emsp;其中，`net1`和`net2`分别表示两个子网络，`torch.cat`在指定维度上拼接张量。输出结果的大小为`torch.Size([2, 36])`，其中第一个维度表示batch_size为2，第二个维度表示输出特征图的通道数为36，正好为两个模型的输出特征图通道数之和（12+24=36）。

In [4]:
import torch.nn as nn
import torch


class Parallel(nn.Module):
    # 自定义并行块
    def __init__(self, net1, net2):
        super().__init__()
        self.net1=net1 # 第一个子网络
        self.net2=net2 # 第二个子网络
        
    def forward(self,X):
        x1= self.net1(X) # 第一个子网络的输出
        x2= self.net2(X) # 第二个子网络的输出
        return torch.cat((x1,x2),dim=1) # 在通道维度上合并输出结果
      
X = torch.rand(2,10) # 输入数据
net = Parallel(nn.Sequential(nn.Linear(10,12),nn.ReLU()), nn.Sequential(nn.Linear(10,24),nn.ReLU())) # 实例化并行网络
output = net(X)
print("模型结构 :", net, sep="\n") # 查看模型结构
print("模型输出的形状 :", output.size()) # 输出结果的大小

模型结构 :
Parallel(
  (net1): Sequential(
    (0): Linear(in_features=10, out_features=12, bias=True)
    (1): ReLU()
  )
  (net2): Sequential(
    (0): Linear(in_features=10, out_features=24, bias=True)
    (1): ReLU()
  )
)
模型输出的形状 : torch.Size([2, 36])


### 练习 5.1.3

假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

**解答：** 

&emsp;&emsp;下面代码定义了一个函数`create_network`，该函数接受四个参数：`num_instances`、`input_size`、`hidden_size` 和 `output_size`，并返回一个Sequential模型。

&emsp;&emsp;其中，该网络模型首先包含 `num_instances` 个相同的线性层，每个线性层有两个子层：一个输入维度为 `input_size`，输出维度为 `hidden_size` 的全连接层，和一个 ReLU 非线性激活层。然后，这 `hidden_size` 个线性层连接在一起作为整个网络的前馈部分。最后，额外添加一个输出层，其输入维度为 `input_size`，输出维度为 `output_size`。

&emsp;&emsp;因此，最终的网络结构是由 `output_size` 个相同的线性层组成的前馈神经网络，每个线性层内部包含一个全连接层和一个ReLU激活层，以及一个独立的输出层。

In [5]:
import torch.nn as nn
import torch


def create_network(num_instances, input_size, hidden_size, output_size):
    # 创建一个线性层
    linear_layer = nn.Sequential(
        nn.Linear(input_size, hidden_size), nn.ReLU(),
        nn.Linear(hidden_size, input_size)
    )
    
    # 创建多个相同结构的实例并连接
    instances = [linear_layer for _ in range(num_instances)]
    network = nn.Sequential(*instances)
    
    # 添加输出层
    output_layer = nn.Linear(input_size, output_size)
    network.add_module("output", output_layer)
    
    return network

# 模型参数
num_instances=3
input_size=10
hidden_size=5
output_size=2

# 示例用法
net = create_network(num_instances=num_instances, input_size=input_size, 
                     hidden_size=hidden_size, output_size=output_size) # 重复3次相同的线形层

# 查看模型结构
print(net)

Sequential(
  (0): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (1): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (2): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (output): Linear(in_features=10, out_features=2, bias=True)
)


## 5.2 参数管理 

### 练习 5.2.1

使用`NestMLP`模型，访问各个层的参数。

 **解答：**

&emsp;&emsp;引用上5.1节中的`NestMLP`模型，可以使用以下代码访问该模型各个层的参数，输出结果将显示每个层对应的参数名称、形状和具体参数。

In [6]:
import torch
import torch.nn as nn


class NestMLP(nn.Module):
    # 5.1节中的NestMLP模型
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

model = NestMLP()

# 访问net层的参数
print('访问net层的参数')
for name, param in model.net.named_parameters():
    print(f"参数名称: {name}, 形状:{param.shape}") # 打印具体参数值所占页面空间较大，读者可按照下面注释的代码自行打印查看。
    # print(f"参数名称: {name}, 形状:{param.shape}, 具体参数值: {param}")

# 访问linear层的参数
print('\n访问linear层的参数')
for name, param in model.linear.named_parameters():
    print(f"参数名称: {name}, 形状:{param.shape}") # 打印具体参数值所占页面空间较大，读者可按照下面注释的代码自行打印查看。
    # print(f"参数名称: {name}, 形状:{param.shape}, 具体参数值: {param}")


访问net层的参数
参数名称: 0.weight, 形状:torch.Size([64, 20])
参数名称: 0.bias, 形状:torch.Size([64])
参数名称: 2.weight, 形状:torch.Size([32, 64])
参数名称: 2.bias, 形状:torch.Size([32])

访问linear层的参数
参数名称: weight, 形状:torch.Size([16, 32])
参数名称: bias, 形状:torch.Size([16])


### 练习 5.2.2

查看初始化模块文档以了解不同的初始化方法。

**解答：**

&emsp;&emsp;通过查看深度学习框架文档，有以下初始化方法 （参考链接：https://pytorch.org/docs/stable/nn.init.html ）
- `torch.nn.init.uniform_(tensor, a=0.0, b=1.0)`：以均匀分布$U(a,b)$初始化输入张量。

- `torch.nn.init.normal_(tensor, mean=0.0, std=1.0)`：以正态分布$N(mean, std^2)$初始化输入张量。

- `torch.nn.init.constant_(tensor, val)`：以一确定数值初始化输入张量。

- `torch.nn.init.ones_(tensor)`：用标量值 1 初始化输入张量。

- `torch.nn.init.zeros_(tensor)`：用标量值 0 初始化输入张量。

- `torch.nn.init.eye_(tensor)`：用单位矩阵初始化二维输入张量。

- `torch.nn.init.xavier_uniform_(tensor, gain=1.0)`：从均匀分布$U(−a, a)$中采样，初始化输入张量，其中$a$的值由如下公式确定

  $$a= gain * \sqrt{\frac{6}{fan_{in}+fan_{out}}}$$
  
  其中$gain$的取值如下表所示
<style> table { margin: auto;} </style>
非线性函数 | gain值
:----:|:----:
Linear/Identity | 1
Conv1D | 1
Conv2D | 1
Conv3D | 1
Sigmoid | 1
Tanh | $\displaystyle\frac{5}{3}$
ReLU | $\sqrt{2}$
Leaky ReLU | $$\sqrt{\frac{2}{1+negative\_slope^2}}$$
SELU | 1 (adaptive)

- `torch.nn.init.xavier_normal_(tensor, gain=1.0)`:从正态分布$N(0,std^2)$中采样，初始化输入张量，其中$std$值由下式确定：

  $$std= gain * \sqrt{\frac{2}{fan_{in}+fan_{out}}}$$

- `torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')`:服从均匀分布$U(−bound, bound)$，其中$bound$值由下式确定

  $$bound= gain * \sqrt{\frac{3}{fan_{mode}}}$$

- `torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')`:服从从正态分布$N(0,std^2)$中采样，其中$std$值由下式确定

  $$std= \frac{gain}{\sqrt{fan_{mode}}}$$
  
- `torch.nn.init.trunc_normal_(tensor, mean=0.0, std=1.0, a=- 2.0, b=2.0)`:用从截断的正态分布中提取的值初始化输入张量。这些值实际上是从正态分布 $N(mean, std^2)$中提取的。

- `torch.nn.init.sparse_(tensor, sparsity, std=0.01)`：将 2D 输入张量初始化为稀疏矩阵，其中非零元素服从正态分布$N(0,0.01)$。

### 练习 5.2.3

构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。

**解答：** 

&emsp;&emsp;在训练过程中，我们每个epoch都打印了每层的参数和梯度。可以看到`shared_fc`层的参数和梯度都是相同的，因为它们共享同一个参数。

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# 模型参数
input_size = 3
hidden_size = 4
output_size = 3
lr = 0.01
epochs = 2

# 构建带有共享参数层的多层感知机
shared_fc = nn.Linear(hidden_size, hidden_size)
MLP = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(),
                    shared_fc, nn.ReLU(),
                    shared_fc, nn.ReLU(),
                    nn.Linear(hidden_size, output_size)
)

# 训练数据
X = torch.randn(1, input_size)
Y = torch.randn(1, output_size)
# 优化器
optimizer = optim.SGD(MLP.parameters(), lr=lr)
# 训练模型
for epoch in range(epochs):
    # 前向传播和计算损失
    Y_pred = MLP(X)
    loss = nn.functional.mse_loss(Y_pred, Y)
    # 反向传播和更新梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 打印每层的参数和梯度
    print(f'Epoch: {epoch}')
    for name, param in MLP.named_parameters():
        print(f"参数名称: {name}, 具体参数值: {param.data[:2]}, 参数梯度: {param.grad[:2]}") # 为了节省页面空间，这里只打印了每个参数和梯度的前两维，读者可以自行调整查看维度
    print(f'Epoch: {epoch}, Loss: {loss.item()}\n')

Epoch: 0
参数名称: 0.weight, 具体参数值: tensor([[ 0.5397,  0.3243, -0.1528],
        [ 0.5680,  0.0952,  0.1176]]), 参数梯度: tensor([[-0., 0., -0.],
        [-0., 0., -0.]])
参数名称: 0.bias, 具体参数值: tensor([-0.4685,  0.5419]), 参数梯度: tensor([0., 0.])
参数名称: 2.weight, 具体参数值: tensor([[ 0.3473, -0.1984,  0.2330, -0.0156],
        [ 0.1252,  0.3434,  0.1973, -0.1091]]), 参数梯度: tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0301, 0.1367, 0.1547]])
参数名称: 2.bias, 具体参数值: tensor([-0.1597,  0.0911]), 参数梯度: tensor([0.0000, 0.3813])
参数名称: 6.weight, 具体参数值: tensor([[-0.4973, -0.1707, -0.2476,  0.2728],
        [ 0.4359, -0.2659,  0.2447,  0.4783]]), 参数梯度: tensor([[-0.0000, -0.1695, -0.4040, -0.2494],
        [ 0.0000,  0.0453,  0.1081,  0.0667]])
参数名称: 6.bias, 具体参数值: tensor([ 0.4901, -0.4670]), 参数梯度: tensor([-1.0651,  0.2849])
Epoch: 0, Loss: 1.0691800117492676

Epoch: 1
参数名称: 0.weight, 具体参数值: tensor([[ 0.5397,  0.3243, -0.1528],
        [ 0.5680,  0.0952,  0.1176]]), 参数梯度: tensor([[-0., 0., -0.],
     

可以看出这个过程只有两个共享层的参数和梯度数值是一样的，我们直接对这两层结果进行进一步的确认。

In [8]:
# 检查参数是否相同
print(f"参数是否相同: {MLP[2].weight.data == MLP[4].weight.data}") 
# 确保它们实际上是同一个对象，而不只是有相同的值
MLP[2].weight.data[0, 0] = 100 
print(f"参数是否同时变化: {MLP[2].weight.data[0] == MLP[4].weight.data[0]}") 
print(f"是否时同一个对象: {MLP[2].weight.data.equal(MLP[4].weight.data)}")


# 检查参数是否相同
print(f"梯度是否相同: {MLP[2].weight.grad == MLP[4].weight.grad}") 
# 确保它们实际上是同一个对象，而不只是有相同的值
MLP[2].weight.grad[0, 0] = 100 
print(f"梯度是否同时变化: {MLP[2].weight.grad[0] == MLP[4].weight.grad[0]}") 
print(f"是否时同一个对象: {MLP[2].weight.grad.equal(MLP[4].weight.grad)}")

参数是否相同: tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
参数是否同时变化: tensor([True, True, True, True])
是否时同一个对象: True
梯度是否相同: tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
梯度是否同时变化: tensor([True, True, True, True])
是否时同一个对象: True


### 练习 5.2.4

为什么共享参数是个好主意？

**解答：** 

&emsp;&emsp;1. 节约内存：共享参数可以减少模型中需要存储的参数数量，从而减少内存占用。

&emsp;&emsp;2. 加速收敛：共享参数可以让模型更加稳定，加速收敛。

&emsp;&emsp;3. 提高泛化能力：共享参数可以帮助模型更好地捕捉数据中的共性，提高模型的泛化能力。

&emsp;&emsp;4. 加强模型的可解释性：共享参数可以让模型更加简洁明了，加强模型的可解释性。 

## 5.3 延后初始化 

### 练习 5.3.1 

如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，会发生什么？是否立即进行初始化？

**解答：** 

&emsp;&emsp;如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，可以正常运行。第一层会立即初始化，但其他层是直到数据第一次通过模型传递才会初始化。

In [2]:
import torch
from torch import nn

"""延后初始化"""
# 定义模型
net = nn.Sequential(nn.Linear(3, 4), nn.ReLU(), nn.LazyLinear(8))
# 尚未初始化

print("--- 数据未通过模型 ---")
print("第一层权重：", net[0].weight)
print("其它层的权重：", net[2].weight)

X = torch.rand(2, 3)
net(X)
print("--- 数据第一次通过模型 ---")
print("第一层权重：", net[0].weight)
print("其它层的权重：", net[2].weight)

--- 数据未通过模型 ---
第一层权重： Parameter containing:
tensor([[-0.3413, -0.5182, -0.3627],
        [ 0.2411, -0.4914,  0.0811],
        [ 0.0203,  0.4713,  0.1287],
        [-0.1224,  0.2020,  0.0286]], requires_grad=True)
其它层的权重： <UninitializedParameter>
--- 数据第一次通过模型 ---
第一层权重： Parameter containing:
tensor([[-0.3413, -0.5182, -0.3627],
        [ 0.2411, -0.4914,  0.0811],
        [ 0.0203,  0.4713,  0.1287],
        [-0.1224,  0.2020,  0.0286]], requires_grad=True)
其它层的权重： Parameter containing:
tensor([[ 0.0142,  0.1109, -0.2118,  0.2039],
        [-0.3138, -0.1928,  0.2217, -0.0317],
        [ 0.3973,  0.2929,  0.0405,  0.3447],
        [-0.0141,  0.0498, -0.4321, -0.2023],
        [ 0.2127,  0.0106, -0.3949, -0.2392],
        [-0.3214,  0.4515, -0.0378,  0.2473],
        [-0.2836,  0.3446,  0.0734, -0.4093],
        [ 0.4049,  0.1875,  0.0278,  0.4894]], requires_grad=True)


### 练习 5.3.2

如果指定了不匹配的维度会发生什么？

**解答：** 

&emsp;&emsp;如果指定了不匹配的维度，会由于矩阵乘法时维度不匹配而报错。在下面的代码中便指定了不匹配的维度。

&emsp;&emsp;由于第一层 nn.Linear(20, 256) 的输入维度为 20，所以输入数据 X 的最后一维必须为 20 才能与该层的权重矩阵相乘。

In [26]:
import torch
from torch import nn

# 定义模型
net = nn.Sequential(
    nn.Linear(20, 256), nn.ReLU(),
    nn.LazyLinear(128), nn.ReLU(),
    nn.LazyLinear(10))

X = torch.rand(2, 10)
try:
    net(X)
except Exception as e:
    print(e)

mat1 and mat2 shapes cannot be multiplied (2x10 and 20x256)


### 练习 5.3.3 

如果输入具有不同的维度，需要做什么？提示：查看参数绑定的相关内容。

**解答：**

&emsp;&emsp;如果输入维度比指定维度小，可以考虑使用 padding 填充；如果输入维度比指定维度大，可以考虑用 PCA 等降维方法，将维度降至指定维度。对于不同的维度，还可以添加一个额外的线性层，并将第一个线性层的权重与该层的权重绑定在一起。这样就可以解决维度不匹配的问题，并且保持模型的权重不变。注意，在下面的代码中，我们假设第一个线性层的偏置项为零，因此不需要对其进行参数绑定。

In [11]:
import torch
import torch.nn as nn

# 定义模型
net = nn.Sequential(
    nn.Linear(20, 256), nn.ReLU(),
    nn.Linear(256, 128), nn.ReLU(),
    nn.Linear(128, 10))

X = torch.rand(2, 10)

try:
    net(X)
except Exception as e:
    print(e)

# 添加额外的线性层
extra_layer = nn.Linear(10, 256)
print(f"第一个线性层的维度：{net[0].weight.shape}")
print(f"额外的线性层的维度：{extra_layer.weight.shape}")

# 将第一个线性层与额外的线性层的权重进行绑定
net[0].weight = extra_layer.weight
print(f"第一个线性层的新维度：{net[0].weight.shape}")

# 使用新的输入（维度为20）调用模型
net(X)

mat1 and mat2 shapes cannot be multiplied (2x10 and 20x256)
第一个线性层的维度：torch.Size([256, 20])
额外的线性层的维度：torch.Size([256, 10])
第一个线性层的新维度：torch.Size([256, 10])


tensor([[-0.0278,  0.0480,  0.0217, -0.0622, -0.0822,  0.0875,  0.0086, -0.0151,
          0.0935,  0.0059],
        [-0.0331,  0.0946, -0.0093, -0.0583, -0.0625,  0.0261,  0.0248, -0.0805,
          0.1055,  0.0188]], grad_fn=<AddmmBackward0>)

## 5.4 自定义层 

### 练习 5.4.1 

设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。

**解答：** 

&emsp;&emsp;这个公式表示一个线性变换，将输入张量$x$中所有可能的二元组$(x_i,x_j)$进行组合，并对它们进行加权求和。其中，$W_{ijk}$表示权重张量中第$i,j,k$个元素的值。具体而言，该公式计算了输入张量$x$中所有二元组$(x_i, x_j)$对应的特征向量$u_{ij}$：


$$
u_{ij} = x_i \cdot x_j
$$


&emsp;&emsp;然后，根据权重张量$W$中的权重$W_{ijk}$，对所有特征向量$u_{ij}$进行线性组合，得到输出向量$y_k$为：


$$
y_k = \sum_{i,j} W_{ijk} u_{ij} = \sum_{i,j} W_{ijk} x_i x_j
$$


&emsp;&emsp;该操作可以被视为一种降维操作，将高维输入$x$映射到低维输出空间$y$中。

In [12]:
import torch
import torch.nn as nn

class TensorReduction(nn.Module):
    def __init__(self, dim1, dim2):
        super(TensorReduction, self).__init__()
        # 定义一个可训练的权重参数，维度为(dim2, dim1, dim1)
        self.weight = nn.Parameter(torch.rand(dim2, dim1, dim1))

    def forward(self, X):
        # 初始化一个全零张量，大小为(X.shape[0], self.weight.shape[0])
        Y = torch.zeros(X.shape[0], self.weight.shape[0])
        for k in range(self.weight.shape[0]):
            # 计算temp = X @ weight[k] @ X^T
            temp = torch.matmul(X, self.weight[k]) @ X.T
            # 取temp的对角线元素，存入Y[:, k]
            Y[:, k] = temp.diagonal()
        return Y

# 创建一个TensorReduction层，dim1=10, dim2=5
layer = TensorReduction(10, 5)
# 创建一个大小为(2, 10)的张量X
X = torch.rand(2, 10)
# 对layer(X)进行前向传播，返回一个大小为(2, 5)的张量
layer(X).shape

torch.Size([2, 5])

### 练习 5.4.2 

设计一个返回输入数据的傅立叶系数前半部分的层。

**解答：** 

&emsp;&emsp;根据[维基百科](https://en.wikipedia.org/wiki/Fourier_series)中:
> &emsp;&emsp;傅里叶级数是将任意周期函数表示为一组正弦和余弦函数的无限级数的方法。假设$f(x)$是在区间$[-L,L]$中定义的一个函数，其周期为$2L$，则其傅里叶级数可表示为：
> 
> $$f(x) = \frac{a_0}{2} + \sum_{n=1}^{\infty} [a_n \cos(\frac{n\pi x}{L}) + b_n \sin(\frac{n\pi x}{L})]$$
> 
> &emsp;&emsp;其中，系数$a_0, a_n$和$b_n$可以通过以下公式计算得出：
> 
> $$a_0 = \frac{1}{2L} \int_{-L}^{L} f(x)dx$$
> 
> $$a_n = \frac{1}{L} \int_{-L}^{L} f(x) \cos(\frac{n\pi x}{L}) dx, n>0$$
> 
> $$b_n = \frac{1}{L} \int_{-L}^{L} f(x) \sin(\frac{n\pi x}{L}) dx, n>0$$
> 
> &emsp;&emsp;系数$a_n$和$b_n$实际上是$f(x)$与$\cos(\frac{n\pi x}{L})$和$\sin(\frac{n\pi x}{L})$的内积，而$a_0$是$f(x)$平均值的一半。

&emsp;&emsp;在`torch`中有相应的函数可以轻松的实现傅里叶级数，如下代码所示：

In [13]:
import torch.nn as nn
import torch.fft as fft

class FourierLayer(nn.Module):
    def __init__(self):
        super(FourierLayer, self).__init__()

    def forward(self, x):
        # 对输入的张量 x 进行快速傅里叶变换
        x = fft.fftn(x)
        # 取出第三个维度的前半部分，即去掉直流分量和镜像分量
        x = x[:, :, :x.shape[2] // 2]
        # 返回处理后的张量
        return x

# 创建一个随机数值为 [0, 1) 的形状为 (1, 2, 5) 的张量 X
X = torch.rand(1, 2, 5)
# 实例化一个 FourierLayer 的网络对象 net
net = FourierLayer()
# 将 X 输入到网络 net 中进行前向计算，并输出结果
net(X)

tensor([[[ 4.6075+0.0000j, -0.6063+1.0520j],
         [ 0.5813+0.0000j,  0.4290-0.5440j]]])

## 5.5 读写文件 

###  练习 5.5.1

即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？

**解答：** 

&emsp;&emsp;1. 加速模型训练：存储模型参数可以避免每次重新训练模型时需要重复计算之前已经计算过的权重和偏置。

&emsp;&emsp;2. 节省内存空间：保存模型参数比保存完整的模型文件更加节省内存空间，这在处理大型模型或使用内存受限设备时尤为重要。

&emsp;&emsp;3. 便于共享和复现：存储模型参数可以方便地共享和复现已经训练好的模型，其他人可以直接加载这些参数并使用它们进行预测或微调。

&emsp;&emsp;4. 便于调试和分析：通过检查模型参数，可以更容易地诊断模型中存在的问题，并对其进行调整和优化。

### 练习 5.5.2

假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

**解答：** 

&emsp;&emsp;使用保存模型某层参数的办法，保存网络的前两层，然后再加载到新的网络中使用。

In [14]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):             # 定义 MLP 类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)   # 定义隐藏层层，输入尺寸为 20，输出尺寸为 256
        self.output = nn.Linear(256, 10)   # 定义输出层，输入尺寸为 256，输出尺寸为 10

    def forward(self, x):          # 定义前向传播函数
        return self.output(F.relu(self.hidden(x)))  # 使用 ReLU 激活函数，计算隐藏层和输出层的输出

class MLP_new(nn.Module):             # 定义 MLP 类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)   # 定义隐藏层层，输入尺寸为 20，输出尺寸为 256
        self.output = nn.Linear(256, 10)   # 定义输出层，输入尺寸为 256，输出尺寸为 10

    def forward(self, x):          # 定义前向传播函数
        return self.output(F.relu(self.hidden(x)))  # 使用 ReLU 激活函数，计算隐藏层和输出层的输出

net = MLP()                       # 创建 MLP 的实例
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')  # 将隐藏层的参数保存到文件中
clone = MLP_new()                     # 创建另一个 MLP 的实例
clone.hidden.load_state_dict(torch.load('mlp.hidden.params'))  # 加载已保存的参数到克隆实例的隐藏层中
print(clone.hidden.weight == net.hidden.weight)  # 比较两个 MLP 实例的隐藏层权重是否相等，并输出结果

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])


### 练习 5.5.3

如何同时保存网络架构和参数？需要对架构加上什么限制？

**解答：** 

&emsp;&emsp;在PyTorch中，可以使用`torch.save()`函数同时保存网络架构和参数。为了保存网络架构，需要将模型的结构定义在一个Python类中，并将该类实例化为模型对象。此外，必须确保该类的构造函数不包含任何随机性质的操作，例如`dropout`层的随机丢弃率应该是固定的。

In [15]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):             # 定义 MLP 类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)   # 定义隐藏层层，输入尺寸为 20，输出尺寸为 256
        self.output = nn.Linear(256, 10)   # 定义输出层，输入尺寸为 256，输出尺寸为 10

    def forward(self, x):          # 定义前向传播函数
        return self.output(F.relu(self.hidden(x)))  # 使用 ReLU 激活函数，计算隐藏层和输出层的输出

net = MLP()

# 存储模型
torch.save(net.state_dict(), 'model.pt')

# 导入模型
model = torch.load('model.pt')
model

OrderedDict([('hidden.weight',
              tensor([[ 0.2060,  0.0169,  0.0780,  ...,  0.0201,  0.1381,  0.0925],
                      [-0.1499, -0.2105,  0.0278,  ..., -0.0910, -0.0794, -0.0869],
                      [ 0.0719, -0.1977, -0.2139,  ...,  0.2095, -0.0749,  0.1598],
                      ...,
                      [-0.1494, -0.0726,  0.1377,  ...,  0.2142,  0.1904,  0.1849],
                      [-0.1231,  0.0370,  0.1983,  ..., -0.0256,  0.1969,  0.0533],
                      [-0.1931, -0.1350, -0.0889,  ...,  0.0144,  0.0857,  0.0599]])),
             ('hidden.bias',
              tensor([ 0.1722,  0.2041, -0.1814,  0.0245, -0.1959, -0.1975, -0.1380, -0.0278,
                       0.1295,  0.2230,  0.0138, -0.2027,  0.1607,  0.1151,  0.0499,  0.1869,
                      -0.0012, -0.0401, -0.0526,  0.1953, -0.1076,  0.1268, -0.1275,  0.0566,
                       0.1308, -0.1600,  0.0856,  0.1625, -0.0309, -0.0318,  0.0066,  0.1644,
                       0.2228,

## 5.6 GPU 

### 练习 5.6.1 

尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？

**解答：** 

&emsp;&emsp;计算量很大的任务：使用GPU速度明显更快。

&emsp;&emsp;计算量很小的任务：CPU速度可能更快，因为数据传输到GPU需要时间。

In [16]:
import time
import torch

# 计算量较大的任务
X = torch.rand((10000, 10000))
Y = X.cuda(0)
time_start = time.time()
Z = torch.mm(X, X)
time_end = time.time()
print(f'cpu time cost: {round((time_end - time_start) * 1000, 2)}ms')
time_start = time.time()
Z = torch.mm(Y, Y)
time_end = time.time()
print(f'gpu time cost: {round((time_end - time_start) * 1000, 2)}ms')

# 计算量很小的任务
X = torch.rand((100, 100))
Y = X.cuda(0)
time_start = time.time()
Z = torch.mm(X, X)
time_end = time.time()
print(f'cpu time cost: {round((time_end - time_start) * 1000)}ms')
time_start = time.time()
Z = torch.mm(Y, Y)
time_end = time.time()
print(f'gpu time cost: {round((time_end - time_start) * 1000)}ms')

cpu time cost: 2131.88ms
gpu time cost: 37.21ms
cpu time cost: 0ms
gpu time cost: 0ms


### 练习 5.6.2

我们应该如何在GPU上读写模型参数？

**解答：** 

&emsp;&emsp;使用`net.to(device)`将模型迁移到GPU上，然后再按照之前的方法读写参数。

In [17]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):             # 定义 MLP 类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)   # 定义隐藏层层，输入尺寸为 20，输出尺寸为 256
        self.output = nn.Linear(256, 10)   # 定义输出层，输入尺寸为 256，输出尺寸为 10

    def forward(self, x):          # 定义前向传播函数
        return self.output(F.relu(self.hidden(x)))  # 使用 ReLU 激活函数，计算隐藏层和输出层的输出

# 选择GPU，没有GPU就选CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 创建模型实例对象
net = MLP()
# 将模型参数传输到GPU上
net.to(device)
# 访问模型参数
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.0496,  0.1606,  0.0861,  ..., -0.1966, -0.1275, -0.1557],
                      [ 0.0648,  0.1355,  0.1571,  ...,  0.0767,  0.0710, -0.1160],
                      [-0.1992,  0.1369,  0.1053,  ...,  0.1187,  0.0040, -0.0608],
                      ...,
                      [-0.0032, -0.1003,  0.1094,  ...,  0.2188, -0.1616, -0.1339],
                      [-0.0660, -0.0040,  0.0260,  ..., -0.0386,  0.1124, -0.1249],
                      [-0.1442,  0.0856,  0.0353,  ...,  0.0307, -0.0522, -0.0307]],
                     device='cuda:0')),
             ('hidden.bias',
              tensor([ 0.0323,  0.1698, -0.0760, -0.0415,  0.0097, -0.1557, -0.1857,  0.1272,
                      -0.0262,  0.1571,  0.1617, -0.1123, -0.0720, -0.0987,  0.1544, -0.0892,
                      -0.0234,  0.0718, -0.1665, -0.1429, -0.0378, -0.1961,  0.1036, -0.1972,
                       0.0961,  0.0816, -0.1794, -0.0917, -0.2164,  0.1339,  0.0810,  

### 练习 5.6.3 

测量计算1000个$100 \times 100$矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。

**解答:** 

&emsp;&emsp;中文版翻译有点问题，英文原版这句话是：

>Measure the time it takes to compute 1000 matrix-matrix multiplications of $100×100$ matrices and log the Frobenius norm of the output matrix one result at a time vs. keeping a log on the GPU and transferring only the final result.

&emsp;&emsp;所以这道题的本质还是希望我们做个比较。

&emsp;&emsp;实验一：仅记录1000次$100×100$矩阵相乘所用的时间，不需要打印Frobenius范数。

&emsp;&emsp;实验二：记录1000次$100×100$矩阵相乘所用的时间，并打印Frobenius范数。

In [18]:
import torch
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 生成随机矩阵
matrices = [torch.randn(100, 100).to(device) for i in range(1000)]

# 实验一：计算时间
start_time_1 = time.time()
for i in range(1000):
    result = torch.mm(matrices[i], matrices[i].t())
    frobenius_norm = torch.norm(result)
#     print(frobenius_norm)
end_time_1 = time.time()
print("Time taken:", end_time_1 - start_time_1)

# 实验二：计算时间
start_time_2 = time.time()
for i in range(1000):
    result = torch.mm(matrices[i], matrices[i].t())
    frobenius_norm = torch.norm(result)
    # print(frobenius_norm)
end_time_2 = time.time()
print("Time taken:", end_time_2 - start_time_2)

print(f'实验一消耗时间：{end_time_1 - start_time_1}，实验二消耗时间：{end_time_2 - start_time_2}')

Time taken: 0.1120142936706543
Time taken: 0.09748530387878418
实验一消耗时间：0.1120142936706543，实验二消耗时间：0.09748530387878418


### 练习 5.6.4

测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：应该看到近乎线性的缩放。

**解答：** 

&emsp;&emsp;执行两个矩阵乘法并行在两个GPU上所需的时间通常会比在单个GPU上按顺序执行这两个操作要快得多。但实际的时间取决于矩阵的大小、硬件配置和算法实现。

&emsp;&emsp;但由于笔者只有一张卡，所以只做了在单个GPU上顺序执行两个矩阵乘法的实验。

In [19]:
import torch
import time

# 创建两个随机矩阵
a = torch.randn(10000, 10000).cuda()
b = torch.randn(10000, 10000).cuda()

# 顺序计算
start_time = time.time()
c1 = torch.matmul(a, b)
c2 = torch.matmul(a, b)
end_time = time.time()
sequential_time = end_time - start_time

print(f"Sequential time: {sequential_time:.8f} seconds")

Sequential time: 0.00903034 seconds
